# **Google Drive credentials**

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# **Loading libraries**

In [0]:
!pip install unidecode

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from wordcloud import WordCloud, ImageColorGenerator
from sklearn.linear_model import LogisticRegression
from unicodedata import normalize
from string import punctuation
from ast import literal_eval
from datetime import date
from nltk import tokenize
from nltk import ngrams
from PIL import Image

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import unidecode
import warnings
import nltk
nltk.download('stopwords')
nltk.download('rslp')

warnings.filterwarnings(action='once')
plt.rcParams["figure.figsize"] = [16,9]


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package rslp to /root/nltk_data...
[nltk_data]   Package rslp is already up-to-date!


# **Loading CSV**

In [0]:
users_data = pd.read_csv("/content/drive/My Drive/Eurotec - CRM/users_data.csv", 
                    sep=";").set_index('id')
user_types_data = pd.read_csv("/content/drive/My Drive/Eurotec - CRM/user_types_data.csv", 
                    sep=";").set_index('id')
euroservices_data = pd.read_csv("/content/drive/My Drive/Eurotec - CRM/euroservices_data.csv", 
                    sep=";").set_index('id')
euroservice_types_data = pd.read_csv("/content/drive/My Drive/Eurotec - CRM/euroservice_types_data.csv", 
                    sep=";").set_index('id')
units_data = pd.read_csv("/content/drive/My Drive/Eurotec - CRM/units_data.csv", 
                    sep=";").set_index('id')
groups_data = pd.read_csv("/content/drive/My Drive/Eurotec - CRM/groups_data.csv", 
                    sep=";").set_index('id')
eurotec_mask = np.array(Image.open("/content/drive/My Drive/Eurotec - CRM/eurotec_logo.png"))

# **Usuários por tipo de usuário**

In [0]:
users_data['user_type'] = user_types_data.loc[users_data['user_type_id']]['name'].values

ax = sns.catplot(y='user_type', kind='count', data = users_data,orient="h", height=6, aspect=11.7/6)
ax.fig.suptitle('Relação tipo de usuário X Quantidade')
ax.set_axis_labels("Quantidade", "Tipo de Usuário")

# **Euroservice por tipo de Euroservice**


In [0]:
euroservices_data['euroservice_type'] = euroservice_types_data.loc[euroservices_data['euroservice_type_id']]['name'].values

# Filter euroservice data to after 2019 only
euroservices_data = euroservices_data[euroservices_data['at_start'] > '2018-12-31']

ax = sns.catplot(y='euroservice_type', kind='count', data = euroservices_data,orient="h", height=6, aspect=11.7/6)
ax.fig.suptitle('Relação tipo de Euroservice X Quantidade')
ax.set_axis_labels("Quantidade", "Tipo de Euroservice")

# **Euroservice por Grupo**

In [0]:
 # groups_data.loc[units_data.loc[72]['group_id']]
euroservices_data['unit'] = units_data.loc[euroservices_data['unit_id']]['name'].values
euroservices_data['group'] = groups_data.loc[units_data.loc[euroservices_data['unit_id']]['group_id'].values]['name'].values

filter_mask = euroservices_data['group'] == 'Eurotec'

ax = sns.catplot(y='group', kind='count', data = euroservices_data[~filter_mask], 
                 orient="h", height=6, aspect=11.7/6,
                 order = euroservices_data['group'].value_counts().index)
ax.fig.suptitle('Relação de Euroservice por Grupo X Quantidade')
ax.set_axis_labels("Quantidade", "Grupo")

In [0]:
euroservices_data_without_eurotec = euroservices_data[~filter_mask]
ax = sns.countplot(y = "group", 
                   data = euroservices_data_without_eurotec,
                   order = euroservices_data_without_eurotec['group'].value_counts().index)
plt.suptitle('Relação de Euroservice por Grupo X Quantidade')
plt.xlabel('Quantidade')
plt.ylabel('Grupo')

total = len(euroservices_data_without_eurotec['group'])
for p in ax.patches:
        percentage = '{:.0f} - {:.2f}%'.format(p.get_width(), 100 * p.get_width()/total)
        x = p.get_x() + p.get_width() + 0.5
        y = p.get_y() + p.get_height()/1.5
        ax.annotate(percentage, (x, y))

plt.show()

# **Euroservice por Usuário**

#### **Transforma a coluna User_ids em array**

In [0]:
euroservices_data['user_ids'] = euroservices_data['user_ids'].apply(literal_eval)
euroservices_data['user_id'] = euroservices_data['user_id'].fillna(0).astype(int)

#### **Dataframe usuários com Euroservices**

In [0]:
users_euroservice_data_temp = pd.DataFrame(columns = ['users_ids', 'euroservice_id', 'user_id']) 
users_euroservice_data_temp['users_ids'] = euroservices_data['user_ids'].fillna(0)
users_euroservice_data_temp['user_id'] = euroservices_data['user_id']
users_euroservice_data_temp['euroservice_id'] = euroservices_data.index

## Limpa a coluna e remove duplicatas
for index, row in users_euroservice_data_temp.iterrows():
    row['users_ids'].append(row['user_id'])

users_euroservice_data_temp = users_euroservice_data_temp.drop(columns='user_id')
for index, row in users_euroservice_data_temp.iterrows():
    users_euroservice_data_temp.at[index,'users_ids'] = pd.unique(row['users_ids']).tolist()

## Divide agora as linhas por user_id
users_euroservice_data = users_euroservice_data_temp.explode('users_ids').reset_index().drop('id', axis=1).rename(columns={'users_ids': 'user_id'})
users_euroservice_data = users_euroservice_data.drop(users_euroservice_data.query("user_id == 0").index)
users_euroservice_data['user_name'] = users_data.loc[users_euroservice_data['user_id']]['name'].values

#### **Plotagem do gráfico**

In [0]:
ax = sns.countplot(y = "user_name", 
                   data = users_euroservice_data,
                   order = users_euroservice_data['user_name'].value_counts().index)

plt.suptitle('Relação de Euroservice por Usuário X Quantidade')
plt.xlabel('Quantidade')
plt.ylabel('Usuário')
total = len(users_euroservice_data['user_name'])
for p in ax.patches:
        percentage = '{:.0f} - {:.2f}%'.format(p.get_width(), 100 * p.get_width()/total)
        x = p.get_x() + p.get_width() + 0.5
        y = p.get_y() + p.get_height()/1.5
        ax.annotate(percentage, (x, y))
plt.show()

# **Euroservices com mais de um Usuário**

In [0]:
euroservices_with_more_than_one_user = pd.concat(g for _, g in users_euroservice_data.groupby("euroservice_id") if len(g) > 1)

In [0]:
euroservices_with_duplications = pd.DataFrame([euroservices_data['unit_id'].count(), euroservices_with_more_than_one_user['euroservice_id'].value_counts().count()], columns = ['count'])

In [0]:
ax = euroservices_with_duplications.plot.bar(title="Euroservices - 2019", legend=False, rot=0)
y_pos = np.arange(len(euroservices_with_duplications['count']))
plt.xticks(y_pos, ['Euroservice', 'Euroservice com mais de 1 técnico'])
plt.ylabel('Quantidade')
total = euroservices_with_duplications['count'].sum()
for p in ax.patches:
  percentage = '{:.0f} - {:.2f}%'.format(p.get_height(), 100 * p.get_height()/total)
  ax.annotate(percentage, (p.get_x() + p.get_width() / 2., p.get_height()), ha='center', va='center', xytext=(0, 10), textcoords='offset points')

# **Trabalhando com os Dados de Euroservice**

## Tempo médio para cada Euroservice

#### Criação de coluna de tempo total de Euroservice

In [0]:
euroservices_data['total_time'] = pd.to_datetime(euroservices_data['at_end']) - pd.to_datetime(euroservices_data['at_start'])
euroservices_data['at_start'] = pd.to_datetime(euroservices_data['at_start'])
euroservices_data['at_end'] = pd.to_datetime(euroservices_data['at_end'])

#### Plotagem do Gráfico

In [0]:
tz_info = euroservices_data.iloc[0].at_start.tzinfo
euroservices_data.sort_values(by=['at_start'], inplace=True)
euroservices_datetime_data = euroservices_data[euroservices_data['total_time'] > '0 days']
for month in range(1, 12):
  df_plot = euroservices_datetime_data[euroservices_datetime_data['at_start'].map(lambda x: x.month) == month]
  fig, ax = plt.subplots(figsize=(12,8))
  ax.plot(df_plot['at_start'],
          df_plot['total_time'])
  ax.set(xlabel="Data",   
        ylabel="Dias",
        title="Tempo médio de Euroservice")
  plt.show()

In [0]:
euroservices_datetime_data[]

In [0]:
euroservices_datetime_data[euroservices_datetime_data['at_start'].map(lambda x: x.month) == 6]['total_time']
euroservices_datetime_data.loc[2379]

### Quantidade de Euroservices com mais de 1 dia e menos de 1 dia

In [0]:
euroservices_data[euroservices_data['total_time'] > '1 days']

## **Criar bag of words / word cloud na busca de recorrência de problemas**

### **Texto em array**

In [0]:
text = euroservices_data['description'].apply(lambda x: normalize('NFKD', x).encode('ASCII', 'ignore').decode('ASCII'))
all_words = ' '.join([text for text in text])

### **Método de geração de Word Cloud**

In [0]:
def word_cloud(all_words):
  word_cloud = WordCloud( background_color = "white",
                          collocations = False,
                          mode = "RGB",
                          mask=eurotec_mask,
                          normalize_plurals = True,
                          random_state = 42).generate(all_words)
  image_colors = ImageColorGenerator(eurotec_mask)
  plt.figure(figsize=(20,20))
  plt.imshow(word_cloud.recolor(color_func=image_colors), interpolation="bilinear")
  plt.axis('off')
  plt.figure()
  plt.show()



### **Método de Classificação do texto**

In [0]:
def text_classify(text):
    vectorize = TfidfVectorizer(lowercase=False, ngram_range = (1,2))
    bag_of_words = vectorize.fit_transform(text)
    train, test, train_class, test_class = train_test_split(bag_of_words,
                                                              text,
                                                              random_state = 42)
    logistic_regression = LogisticRegression(solver = "lbfgs")
    logistic_regression.fit(train, train_class)
    return logistic_regression.score(test, test_class)

classification = pd.DataFrame({ text_classify(text) }, columns = ["Classificação"])

### **Plotagem de Diagrama de Pareto**

In [0]:
def pareto(all_words, n):
    space_token = tokenize.WhitespaceTokenizer()
    token_text = space_token.tokenize(all_words)
    frequency = nltk.FreqDist(token_text)
    df_frequency = pd.DataFrame({"Palavra": list(frequency.keys()),
                                   "Frequência": list(frequency.values())})
    df_frequency = df_frequency.nlargest(columns = "Frequência", n = n)
    plt.figure(figsize=(12,8))
    ax = sns.barplot(data = df_frequency, x = "Palavra", y = "Frequência")
    ax.set(ylabel = "Contagem")
    plt.show()
    
pareto(all_words, 10)

### **Tratamento dos dados (Palavras utilizadas nas descrições)**


#### **Remoção de palavras que não adicionam contexto, pontuação e acentuação**

In [0]:
useless_words = nltk.corpus.stopwords.words("portuguese")

punctuation_token = tokenize.WordPunctTokenizer()

punctuation_list = list()
for p in punctuation:
    punctuation_list.append(p)

punctuation_stopwords = punctuation_list + useless_words
punctuation_stopwords = [unidecode.unidecode(text) for text in punctuation_stopwords]

processed_text = list()
for word in text:
    word = word.lower()
    new_word = list()
    text_word = punctuation_token.tokenize(word)
    for another_word in text_word:
        if another_word not in punctuation_stopwords:
            new_word.append(another_word)
    processed_text.append(' '.join(new_word))

all_words_processed = ' '.join([processed_text for processed_text in processed_text]) 

### **Plotagem dos dados tratados**

In [0]:
pareto(all_words_processed, 10)

In [0]:
word_cloud(all_words_processed)

### **Histórico de confiança dos dados**

In [0]:
classification = classification.append(pd.DataFrame({ text_classify(processed_text) }, columns = ["Classificação"]))
sns.lineplot(x="index",  y="Classificação", data=classification.reset_index(drop=True).reset_index())
plt.suptitle("Timeline de classificações de Word Cloud")